In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [15]:
#get Basic Probability Assignment
def getMass(sample, accuracy):
    bp = [[0,0,0,0],[0,0,0,0]]
    for i in range(len(sample)):
        if sample[i] == 'S':
            bp[0][i]=accuracy[i]
        else:
            bp[0][i]=1-accuracy[i]
        if sample[i] == 'M':
            bp[1][i]=accuracy[i]
        else:
            bp[1][i]=1-accuracy[i]   
    return bp

In [16]:
#get Normalized constant
def getK(bp):
    k=bp[0][0]*bp[0][1]+bp[1][0]*bp[1][1]
    return k

In [17]:
#Data fusion according to Dempster combination rules
def getDempster(k, bp):
    m = bp[0]*bp[1]/k
    return m

In [18]:
def doDempsterShafer(sample,accuracy,anum):
    #get Basic Probability
    BasicProbability = getMass(sample,accuracy)
    #print(BasicProbability)
    mS = 0
    mM = 0
    k =0
    bp_temp = [[0,0],[0,0]]
    #Ds Fusion
    for m in range(anum-1):
        if m==0 :
            bp_temp[0][0] = BasicProbability[0][0]
            bp_temp[1][0] = BasicProbability[1][0]
        else:
            bp_temp[0][0] = mS
            bp_temp[1][0] = mM
        bp_temp[0][1] = BasicProbability[0][m+1]
        bp_temp[1][1] = BasicProbability[1][m+1]
        
        k = getK(bp_temp)
        mS = getDempster(k,bp_temp[0])
        mM = getDempster(k,bp_temp[1])
    if mS>=0.5:
        return 'S'
    else:
        return 'M'

In [3]:
def DempsterShafer_382(accuracylist):
    data = pd.read_excel('middle\\phase_prediction_382.xlsx')
    df1 = data.replace("S","1")
    df2 = df1.replace("M","2")
    df3 = pd.DataFrame(df2[df2.columns[2:]], dtype = np.float64)

    df_LR = df3[df3.columns[0:5]]
    LR_mean = df_LR.mean(axis=1)
    df_LR_avg = pd.DataFrame(LR_mean, columns=['LR_SFS_AVG'])
    df_LR_avg["LR_SFS_AVG"] = np.where(df_LR_avg["LR_SFS_AVG"]<=1.5, "S", "M")
    #df_LR
    #df_LR_avg

    df_NB = df3[df3.columns[5:10]]
    NB_mean = df_NB.mean(axis=1)
    df_NB_avg = pd.DataFrame(NB_mean, columns=['NB_SFS_AVG'])
    df_NB_avg["NB_SFS_AVG"] = np.where(df_NB_avg["NB_SFS_AVG"]<=1.5, "S", "M")
    #df_NB

    df_RF = df3[df3.columns[10:15]]
    RF_mean = df_RF.mean(axis=1)
    df_RF_avg = pd.DataFrame(RF_mean, columns=['RF_SFS_AVG'])
    df_RF_avg["RF_SFS_AVG"] = np.where(df_RF_avg["RF_SFS_AVG"]<=1.5, "S", "M")
    #df_RF
    #df_RF_avg

    df_Sigmoid = df3[df3.columns[25:30]]
    Sigmoid_mean = df_Sigmoid.mean(axis=1)
    df_Sigmoid_avg = pd.DataFrame(Sigmoid_mean, columns=['SVM_Sigmoid_AVG'])
    df_Sigmoid_avg["SVM_Sigmoid_AVG"] = np.where(df_Sigmoid_avg["SVM_Sigmoid_AVG"]<=1.5, "S", "M")
    #df_Sigmoid
    #df_Sigmoid_avg

    data.rename(columns={'Unnamed: 0':'indexs'},inplace=True)
    data_name = data[data.columns[0:2]]
    newdata = pd.concat([data_name,df_LR_avg,df_NB_avg,df_RF_avg,df_Sigmoid_avg],axis=1) 
    
     #LR NB RF sigmoid      
    #Getting the accuracy of each algorithm    
    AccuracySFS =accuracylist[1::2]
    AlgNum = len(AccuracySFS)
    
    data_ds_SFS = []
    formula = list(newdata.iloc[:, 1])
    dataSFS = newdata[['indexs','formula','LR_SFS_AVG','NB_SFS_AVG','RF_SFS_AVG','SVM_Sigmoid_AVG']]
    new_data_SFS = newdata[['LR_SFS_AVG','NB_SFS_AVG','RF_SFS_AVG','SVM_Sigmoid_AVG']].values
    for n in range(382):
        sample_SFS = new_data_SFS[n]
        data_ds_SFS.append(doDempsterShafer(sample_SFS,AccuracySFS,AlgNum))
        #data_ds[n][1] = doDempsterShafer(onesample,AccuracySFS,AlgNum)
    dfsfs = pd.DataFrame(data_ds_SFS, columns=['result_ds_SFS'])
    SFS_data = pd.concat([dataSFS,dfsfs],axis=1) 
    SFS_data.to_excel(excel_writer="ds_result_SFS_382.xlsx",index=False)